In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-txt-classification/sample_submission.csv
/kaggle/input/nlp-txt-classification/train.csv
/kaggle/input/nlp-txt-classification/test.csv


In [3]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00


In [4]:
# Помогает решить ошибки во время импорта "transformers"
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.6 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [5]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import os
import re
import torch
import evaluate
from sklearn import preprocessing
import transformers
from transformers import AutoTokenizer
from transformers import Trainer
from transformers import TrainingArguments
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    default_data_collator   
)

import datasets
from datasets import load_dataset
from datasets import load_dataset_builder
from datasets import Dataset

torch.cuda.is_available()
os.environ["WANDB_DISABLED"] = "true"

stemmer = PorterStemmer()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [6]:
df_train = pd.read_csv('/kaggle/input/nlp-txt-classification/train.csv').dropna()
df_test = pd.read_csv('/kaggle/input/nlp-txt-classification/test.csv')
sample_submission = pd.read_csv('/kaggle/input/nlp-txt-classification/sample_submission.csv')
print('Shape train: {}'.format(df_train.shape))
print('Shape test: {}'.format(df_test.shape))
print('Shape sample_submission: {}'.format(sample_submission.shape))
print('-'*40)
print('Gaps train: {}'.format(df_train.isnull().sum().sum()))
print('Gaps test: {}'.format(df_test.isnull().sum().sum()))
print('Gaps sample_submission: {}'.format(sample_submission.isnull().sum().sum()))

Shape train: (41155, 3)
Shape test: (3798, 2)
Shape sample_submission: (3798, 2)
----------------------------------------
Gaps train: 0
Gaps test: 0
Gaps sample_submission: 0


In [7]:
df_train.sample(5)

,Unnamed: 0,Text,Sentiment
11437,11437,If there was a God this virus would target all...,Extremely Negative
29867,29867,"Amid to covid-19 lockdown, budget aquaman ress...",Neutral
32707,32707,@ate_ted @SenTedCruz Supply &amp; Demand. COVI...,Positive
4854,4854,Empty store shelves in grocery store do to pan...,Negative
1217,1217,President announcement practise social distanc...,Positive


In [8]:
LABELS = df_train['Sentiment'].unique()
LABELS_COUNT = len(df_train['Sentiment'].unique())

# EDA

In [9]:
# Помогает решить ошибки для "wordnet"
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [10]:
def preprocess_text(text, url=False, lower=True, punct=True, stop=True, stem=False, lem=True):

    # Очистка текста от URL-адресов
    if url:
        text = re.sub(r'http\S+', '', text)
    
    # Приведение символов к нижнему регистру
    if lower:
        text = text.lower()

    # Удаление пунктуации
    if punct:
        text = text.translate(str.maketrans("", "", string.punctuation))

    # Удаление стоп-слов
    if stop:
        stop_words = set(stopwords.words('english'))
        words = nltk.word_tokenize(text)
        filtered_words = [word for word in words if word not in stop_words]
        text = " ".join(filtered_words)

    # Стемминг
    if stem:
        porter_stemmer = PorterStemmer()
        words = nltk.word_tokenize(text)
        stemmed_words = [porter_stemmer.stem(word) for word in words]
        text = " ".join(stemmed_words)

    # Лемматизация
    if lem:
        wordnet_lemmatizer = WordNetLemmatizer()
        words = nltk.word_tokenize(text)
        lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in words]
        text = " ".join(lemmatized_words)

    return text

In [11]:
df_train['Text'] = df_train['Text'].apply(preprocess_text)
df_test['Text'] = df_test['Text'].apply(preprocess_text)

In [12]:
df_train.sample(3)

,Unnamed: 0,Text,Sentiment
17344,17344,mondelez hiring 1 000 u employee help maintain...,Positive
25332,25332,enforcement ca consumer privacy act set start ...,Negative
15130,15130,philip ramping production ventilator really ne...,Extremely Negative


In [13]:
# Закодируем признак `Sentiment`
le = preprocessing.LabelEncoder()
le.fit(LABELS)

df_train['Sentiment'] = le.transform(df_train['Sentiment'])
df_train.rename(columns={'Sentiment': 'label'}, inplace=True)

In [14]:
# tokenizer
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=LABELS_COUNT,
                                                           ignore_mismatched_sizes=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [15]:
# Запишем функцию
def tokenize_function(data):
    return tokenizer(data["Text"], truncation=True)

In [16]:
df_test.head()

,id,Text
0,787bc85b-20d4-46d8-84a0-562a2527f684,trending new yorkers encounter empty supermark...
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,couldnt find hand sanitizer fred meyer turned ...
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,find protect loved one coronavirus
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,panic buying hit newyork city anxious shopper ...
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,toiletpaper dunnypaper coronavirus coronavirus...


In [17]:
# Train
train_dt = Dataset.from_pandas(df_train[['Text','label']])
tokenized_train = train_dt.map(tokenize_function, batched=True)
tokenized_train = tokenized_train.remove_columns(["__index_level_0__"])

#test 
test_dt = Dataset.from_pandas(df_test)
tokenized_test = test_dt.map(tokenize_function, batched=True)
tokenized_test = tokenized_test.remove_columns(['id', 'Text'])


data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #padding funct.

  0%|          | 0/42 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [18]:
splitted_data= tokenized_train.train_test_split(test_size=0.2)
splitted_data

DatasetDict({
    train: Dataset({
        features: ['Text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32924
    })
    test: Dataset({
        features: ['Text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8231
    })
})

# Evaluate

In [19]:
# metrics count function
def compute_metrics(eval_preds):
    metric = evaluate.load('accuracy')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
training_args = TrainingArguments(
    "test-trainer",     
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    optim="adamw_torch",
    warmup_steps=600,
    weight_decay=0.01,
    logging_steps=1)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=splitted_data['train'],
    eval_dataset=splitted_data['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
transformers.logging.set_verbosity('CRITICAL')
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.820600,0.727061,0.741465
2,0.420000,0.708806,0.751185
3,0.254600,0.679502,0.778885


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3087, training_loss=0.6911669404115884, metrics={'train_runtime': 1065.5544, 'train_samples_per_second': 92.695, 'train_steps_per_second': 2.897, 'total_flos': 3200484804902088.0, 'train_loss': 0.6911669404115884, 'epoch': 3.0})

## Predict

In [23]:
def get_prediction(text):
    inputs = tokenizer(text, truncation=True,padding=True, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    proba = outputs[0].softmax(1)
    return proba.argmax().item()

In [24]:
trainer.evaluate(tokenized_test)

{'eval_runtime': 10.2267,
 'eval_samples_per_second': 371.381,
 'eval_steps_per_second': 5.867,
 'epoch': 3.0}

In [26]:
predictions = df_test['Text'].apply(lambda text: get_prediction(text))
sample_submission['Sentiment'] = le.inverse_transform(predictions)
sample_submission.sample(6)

,id,Sentiment
1070,52f04637-8897-423a-b546-88c165b31128,Neutral
